
# SVM Hồi quy — Không chuẩn hoá lại (dữ liệu đã được scale ở preprocessing)
**Loại bỏ `country_name`, `country_code`** • **K-fold CV (k=5)** • **L2: LinearSVR** • **L1: SGD (epsilon-insensitive)**

---

## Bước 0 — Mục tiêu tổng quan
- Xây dựng hai phiên bản SVM tuyến tính cho hồi quy:
  - **L2**: `LinearSVR` (ε-SVR với L2 regularization).
  - **L1 (xấp xỉ)**: `SGDRegressor(loss="epsilon_insensitive", penalty="l1")`.
- **Không chuẩn hoá lại features** vì dữ liệu **đã được chuẩn hoá từ bước preprocessing**.
- **Loại bỏ** các cột định danh (`country_name`, `country_code`), chỉ giữ **numeric**.
- **Không chuẩn hoá target `life_expectancy`**.
- Dùng **K-fold CV (k=5)** để chọn siêu tham số; lưu **model `.pkl`** + **metrics CSV** + **CV results**.



## Bước 1 — Thiết lập & Đường dẫn
**Mục tiêu**
- Import thư viện, chỉ định đường dẫn dữ liệu và thư mục lưu output.


In [6]:

import os, joblib, numpy as np, pandas as pd
from sklearn.model_selection import KFold, GridSearchCV, train_test_split
from sklearn.svm import LinearSVR
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Đường dẫn
DATA_DIR = "../data/processed"
X_TRAIN_PATH = f"{DATA_DIR}/X_train.csv"
Y_TRAIN_PATH = f"{DATA_DIR}/y_train.csv"
X_TEST_PATH  = f"{DATA_DIR}/X_test.csv"
Y_TEST_PATH  = f"{DATA_DIR}/y_test.csv"

MODEL_DIR = "../model/2_SVM_regression"
os.makedirs(MODEL_DIR, exist_ok=True)
print("Paths ready.")


Paths ready.



## Bước 2 — Đọc dữ liệu & **Loại bỏ** định danh
**Mục tiêu**
- Bỏ `country_name`, `country_code`; chỉ giữ **numeric**; căn chỉnh test theo train.


In [7]:

# Đọc dữ liệu
X_train = pd.read_csv(X_TRAIN_PATH)
y_train = pd.read_csv(Y_TRAIN_PATH).squeeze("columns")

HAS_TEST = os.path.exists(X_TEST_PATH) and os.path.exists(Y_TEST_PATH)
if HAS_TEST:
    X_test = pd.read_csv(X_TEST_PATH)
    y_test = pd.read_csv(Y_TEST_PATH).squeeze("columns")

# Loại bỏ định danh
id_cols = ['country_name', 'country_code']
X_train = X_train.drop(columns=id_cols, errors='ignore')
if HAS_TEST:
    X_test = X_test.drop(columns=id_cols, errors='ignore')

# Chỉ giữ numeric features
num_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
X_train = X_train[num_cols].copy()
if HAS_TEST:
    X_test = X_test.reindex(columns=num_cols).copy()

# Nếu không có Test sẵn, tách từ Train
if not HAS_TEST:
    X_train, X_test, y_train, y_test = train_test_split(
        X_train, y_train, test_size=0.2, random_state=42
    )
    print("Train:", X_train.shape, "| Val(Test):", X_test.shape)
else:
    print("Train:", X_train.shape, "| Test:", X_test.shape)

# Chuẩn bị y dạng 1D
ytr = np.asarray(y_train, dtype=float).ravel()
yte = np.asarray(y_test, dtype=float).ravel()
Xtr = X_train.values
Xte = X_test.values


Train: (4882, 12) | Test: (543, 12)



## Bước 3 — Hàm tính metrics & lưu CSV
**Mục tiêu**
- Chuẩn hoá cách tính **RMSE/MAE/R²** (không dùng `squared=False`) và lưu kết quả.


In [8]:

def rmse_score(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    return float(np.sqrt(mse))

def metrics_dict(y_true, y_pred):
    return {
        "rmse": rmse_score(y_true, y_pred),
        "mae":  mean_absolute_error(y_true, y_pred),
        "r2":   r2_score(y_true, y_pred),
    }

def save_metrics_csv(path, dct):
    pd.DataFrame([dct]).to_csv(path, index=False)
    return path



## Bước 4 — L2-SVM (`LinearSVR`) + 5-fold CV (không scale lại)
**Mục tiêu**
- GridSearchCV (k=5) chọn `C`, `epsilon`, `loss`; đánh giá Train/Test; lưu model & metrics.


In [9]:

cv5 = KFold(n_splits=5, shuffle=True, random_state=42)

param_grid_l2 = {
    "C": [0.1, 1.0, 10.0],
    "epsilon": [0.05, 0.1, 0.2, 0.5],
    "loss": ["epsilon_insensitive", "squared_epsilon_insensitive"],
}

gs_l2 = GridSearchCV(
    estimator=LinearSVR(random_state=0, max_iter=20000),
    param_grid=param_grid_l2,
    cv=cv5,
    scoring="neg_root_mean_squared_error",
    n_jobs=-1,
)
gs_l2.fit(Xtr, ytr)
svm_l2_best = gs_l2.best_estimator_
print("[L2-SVM] Best params:", gs_l2.best_params_, "| CV best RMSE:", -gs_l2.best_score_)

# Lưu toàn bộ kết quả CV
cv_l2_csv = os.path.join(MODEL_DIR, "svm_l2_cv_results.csv")
pd.DataFrame(gs_l2.cv_results_).to_csv(cv_l2_csv, index=False)

# Đánh giá Train/Test
train_l2 = metrics_dict(ytr, svm_l2_best.predict(Xtr))
test_l2  = metrics_dict(yte, svm_l2_best.predict(Xte))
print("[L2-SVM Train] RMSE={rmse:.4f} | MAE={mae:.4f} | R2={r2:.4f}".format(**train_l2))
print("[L2-SVM Test ] RMSE={rmse:.4f} | MAE={mae:.4f} | R2={r2:.4f}".format(**test_l2))

# Lưu metrics
save_metrics_csv(os.path.join(MODEL_DIR, "svm_l2_train_metrics.csv"), train_l2)
save_metrics_csv(os.path.join(MODEL_DIR, "svm_l2_test_metrics.csv"),  test_l2)

# Lưu model bundle (không có scaler)
bundle_l2 = {"num_cols": list(num_cols), "estimator": svm_l2_best}
pkl_l2 = os.path.join(MODEL_DIR, "svm_l2_linear.pkl")
joblib.dump(bundle_l2, pkl_l2)
print("[L2-SVM] Saved ->", pkl_l2)


[L2-SVM] Best params: {'C': 0.1, 'epsilon': 0.5, 'loss': 'squared_epsilon_insensitive'} | CV best RMSE: 4.456020070958482
[L2-SVM Train] RMSE=4.4567 | MAE=3.3319 | R2=0.7350
[L2-SVM Test ] RMSE=4.2305 | MAE=3.2554 | R2=0.7373
[L2-SVM] Saved -> ../model/2_SVM_regression\svm_l2_linear.pkl



## Bước 5 — L1-SVM (xấp xỉ) bằng `SGDRegressor` (epsilon-insensitive) + 5-fold CV (không scale lại)
**Mục tiêu**
- Xấp xỉ SVM tuyến tính với regularization **L1**; đánh giá Train/Test; lưu model & metrics.


In [10]:

param_grid_l1 = {
    "alpha":   [1e-5, 1e-4, 1e-3],
    "epsilon": [0.05, 0.1, 0.2, 0.5],
}

base_l1 = SGDRegressor(
    loss="epsilon_insensitive", penalty="l1",
    max_iter=5000, tol=1e-3, learning_rate="optimal", random_state=0
)

gs_l1 = GridSearchCV(
    estimator=base_l1,
    param_grid=param_grid_l1,
    cv=cv5,
    scoring="neg_root_mean_squared_error",
    n_jobs=-1,
)
gs_l1.fit(Xtr, ytr)
svm_l1_best = gs_l1.best_estimator_
print("[L1-SVM] Best params:", gs_l1.best_params_, "| CV best RMSE:", -gs_l1.best_score_)

# Lưu toàn bộ kết quả CV
cv_l1_csv = os.path.join(MODEL_DIR, "svm_l1_cv_results.csv")
pd.DataFrame(gs_l1.cv_results_).to_csv(cv_l1_csv, index=False)

# Đánh giá Train/Test
train_l1 = metrics_dict(ytr, svm_l1_best.predict(Xtr))
test_l1  = metrics_dict(yte, svm_l1_best.predict(Xte))
print("[L1-SVM Train] RMSE={rmse:.4f} | MAE={mae:.4f} | R2={r2:.4f}".format(**train_l1))
print("[L1-SVM Test ] RMSE={rmse:.4f} | MAE={mae:.4f} | R2={r2:.4f}".format(**test_l1))

# Lưu metrics
save_metrics_csv(os.path.join(MODEL_DIR, "svm_l1_train_metrics.csv"), train_l1)
save_metrics_csv(os.path.join(MODEL_DIR, "svm_l1_test_metrics.csv"),  test_l1)

# Lưu model bundle (không có scaler)
bundle_l1 = {"num_cols": list(num_cols), "estimator": svm_l1_best}
pkl_l1 = os.path.join(MODEL_DIR, "svm_l1_linear.pkl")
joblib.dump(bundle_l1, pkl_l1)
print("[L1-SVM] Saved ->", pkl_l1)


[L1-SVM] Best params: {'alpha': 0.001, 'epsilon': 0.05} | CV best RMSE: 204.08751168939096
[L1-SVM Train] RMSE=87.6111 | MAE=70.8400 | R2=-101.4220
[L1-SVM Test ] RMSE=88.1094 | MAE=70.7844 | R2=-112.9483
[L1-SVM] Saved -> ../model/2_SVM_regression\svm_l1_linear.pkl
